In [1]:
import numpy as np
import igl
import meshplot as mp
import itertools

In [2]:
# Global Variables
wendlandRadius = 20
polyDegree = 1
resolution = 50
accelerate = True

In [3]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners

def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud (for cat)

In [4]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi = np.dot(pi, np.array([[0, 0, -1], [-1, 0, 0], [0, 1, 0]]))
pi /= 10
ni = igl.per_vertex_normals(pi, v)
ni = np.array([n/np.linalg.norm(n) for n in ni])
mp.plot(pi, shading={"point_size": 8, "point_color": "black"})

D:\Anaconda\envs\GM\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(23.5, -46…

# Using a non-axis-aligned grid (for luigi)

In [5]:
# pi, v = igl.read_triangle_mesh("data/luigi.off")
# pi /= 10
# ni = igl.per_vertex_normals(pi, v)
# ni = np.array([n/np.linalg.norm(n) for n in ni])

# pi = pi - np.mean(pi, axis=0)
# cov = np.cov(pi, rowvar=False)
# eigen_values, eigen_vectors = np.linalg.eig(cov)
# pi = np.dot(pi, eigen_vectors)
# pi = np.dot(pi, np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]]))
# ni = np.dot(ni, eigen_vectors)
# ni = np.dot(ni, np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]]))

# mp.plot(pi, shading={"point_size": 1, "point_color": "black"})

# Implementing a spatial index to accelerate neighbor calculations

In [6]:
def get_index(point):
    return ((point - grid_min) / wendlandRadius).astype(int)

In [7]:
def get_adjacent_index(point):
    # include itself
    index = get_index(point)
#     print(index)
    return [i for i in itertools.product([index[0] - 1, index[0],index[0] + 1], 
                                         [index[1] - 1, index[1],index[1] + 1],
                                         [index[2] - 1, index[2],index[2] + 1])]

In [8]:
bbox_min = np.min(pi, axis=0)
bbox_max = np.max(pi, axis=0)
bbox_diag = np.linalg.norm(bbox_max - bbox_min)

bbox_min_enlarge = bbox_min - 0.1 * bbox_diag
bbox_max_enlarge = bbox_max + 0.1 * bbox_diag
grid_min = bbox_min_enlarge - wendlandRadius
grid_max = bbox_max_enlarge + wendlandRadius
grid_nb = np.ceil((grid_max - grid_min) / wendlandRadius).astype(int)

pi_spatial_index = [[[[] for i in range(grid_nb[2])] for j in range(grid_nb[1])] for k in range(grid_nb[0])]
for i, pii in enumerate(pi):
    index = get_index(pii)
    pi_spatial_index[index[0]][index[1]][index[2]].append(i)

# Setting up the Constraints

In [9]:
def get_adjacent_points(point, points):
    grid_indices = get_adjacent_index(point)
    point_indices = []
    for grid_index in grid_indices:
        if points.shape[0] == pi.shape[0]:
            point_indices += pi_spatial_index[grid_index[0]][grid_index[1]][grid_index[2]]
        elif points.shape[0] == p.shape[0]:
            point_indices += p_spatial_index[grid_index[0]][grid_index[1]][grid_index[2]]
    return point_indices

In [10]:
# Utility function to retreives the index of the colosest point to point in points
def find_closed_point(point, points):
    index = 0
    distance = float('inf')
    if accelerate:
        for i in get_adjacent_points(point, points):
            dis = np.linalg.norm(pi[i] - point)
            if dis < distance:
                distance = dis
                index = i
    else:
        for i, pj in enumerate(points):
            dis = np.linalg.norm(pj - point)
            if dis < distance:
                distance = dis
                index = i
    return index

In [11]:
pi_plus = np.zeros_like(pi)
for i, pii in enumerate(pi):
    eps = 0.01 * bbox_diag
    while True:
        pii_plus = pii + eps * ni[i]
        if i != find_closed_point(pii_plus, pi):
            eps /= 2
        else:
            pi_plus[i] = pii_plus
            break

pi_minus = np.zeros_like(pi)
for i, pii in enumerate(pi):
    eps = 0.01 * bbox_diag
    while True:
        pii_minus = pii - eps * ni[i]
        if i != find_closed_point(pii_minus, pi):
            eps /= 2
        else:
            pi_minus[i] = pii_minus
            break

p = np.vstack((pi, pi_plus, pi_minus))
f = np.hstack(([0] * pi.shape[0], [eps] * pi.shape[0], [-eps] * pi.shape[0]))
# p = pi
# f = np.array([0] * pi.shape[0])

In [12]:
color = np.zeros((f.shape[0], 3))
color[f == 0] = [0, 0, 255]
color[f > 0] = [255, 0, 0]
color[f < 0] = [0, 255, 0]
plot = mp.plot(p, c=color, shading={"point_size": 8}, return_plot=True)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(23.525246…

# Use MLS interpolation to extend to function f

## Create a grid sampling the 3D space

In [13]:
# Generate grid n x n x n
x, T = tet_grid((resolution, resolution, resolution), bbox_min_enlarge, bbox_max_enlarge)

In [14]:
p_spatial_index = [[[[] for i in range(grid_nb[2])] for j in range(grid_nb[1])] for k in range(grid_nb[0])]
for i, pii in enumerate(pi):
    index = get_index(pii)
    p_spatial_index[index[0]][index[1]][index[2]].append(i)
for i, pii in enumerate(pi_plus):
    index = get_index(pii)
    p_spatial_index[index[0]][index[1]][index[2]].append(i + pi.shape[0])
for i, pii in enumerate(pi_minus):
    index = get_index(pii)
    p_spatial_index[index[0]][index[1]][index[2]].append(i + pi.shape[0] * 2)

## MLS Interpolation

In [15]:
def closest_points(point, points, h):
    if accelerate:
        return np.array([i for i in get_adjacent_points(point, points) if np.linalg.norm(points[i] - point) < h])
    else:
        return np.array([i for i, pj in enumerate(points) if np.linalg.norm(pj - point) < h])

In [16]:
def wendland(r):
    return (1 - r / wendlandRadius) ** 4 * (4 * r / wendlandRadius + 1)

In [17]:
def implicit_function(x):
    indices = closest_points(x, p, wendlandRadius)
    coef_num = {0: 1, 1: 4, 2: 10}
#     if np.sum(bbox_min - x > 0) > 0 or np.sum(x - bbox_max > 0) > 0:
#         return 100
    if len(indices) < 2 * coef_num[polyDegree]:
        return 100
    B = np.zeros((len(indices), coef_num[polyDegree]))
    d = np.array([f[i] for i in indices])
    W = np.zeros((len(indices), len(indices)))
    for i, j in enumerate(indices):
        if polyDegree == 0:
            B[i] = np.array([1])
        elif polyDegree == 1:
            B[i] = np.array([1, p[j][0], p[j][1], p[j][2]])
        elif polyDegree == 2:
            B[i] = np.array([1, p[j][0], p[j][1], p[j][2], p[j][0] * p[j][0], p[j][0] * p[j][1],
                          p[j][0] * p[j][2], p[j][1] * p[j][1], p[j][1] * p[j][2], p[j][2] * p[j][2]])
        W[i][i] = wendland(np.linalg.norm(x - p[j]))
    a = np.linalg.solve(np.dot(np.dot(B.T, W), B), np.dot(np.dot(B.T, W), d))
    
    if polyDegree == 0:
        Bx = np.array([1])
    elif polyDegree == 1:
        Bx = np.array([1, x[0], x[1], x[2]])
    elif polyDegree == 2:
        Bx = np.array([1, x[0], x[1], x[2], x[0] * x[0], x[0] * x[1], x[0] * x[2], x[1] * x[1], x[1] * x[2], x[2] * x[2]])
    return np.dot(Bx, a)

In [18]:
#Compute implicit function
fx = np.array([implicit_function(xi) for xi in x])

In [19]:
# Treshold fx to visualize inside outside
ind = np.zeros((len(fx), 3))
ind[fx >= 0] = [255, 0, 0]
ind[fx < 0] = [0, 255, 0]
mp.plot(x, c=ind, shading={"point_size": 8,"width": 800, "height": 800})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(23.500001…

# Extracting the surface

In [20]:
# Marcing tet to extract surface
sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)
sf_index = igl.face_components(sf)
num_dict = {}
for num in sf_index:
    num_dict[num] = num_dict[num] + 1 if num in num_dict.keys() else 1
max_index = max(num_dict,key=num_dict.get)
sf_new = np.array([sf[i] for i, index in enumerate(sf_index) if index == max_index])
mp.plot(sv, sf_new, shading={"wireframe": True} )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(19.395206…

# Optional Task

In [21]:
def implicit_function_optional(x):
    indices = closest_points(x, pi, wendlandRadius)
    coef_num = {0: 1, 1: 4, 2: 10}
    if len(indices) < 2 * coef_num[polyDegree]:
        return 100
    B = np.zeros((len(indices), coef_num[polyDegree]))
    d = np.array([(f[i] + np.dot((x - pi[i]).T, ni[i])) for i in indices])
    W = np.zeros((len(indices), len(indices)))
    for i, j in enumerate(indices):
        if polyDegree == 0:
            B[i] = np.array([1])
        elif polyDegree == 1:
            B[i] = np.array([1, p[j][0], p[j][1], p[j][2]])
        elif polyDegree == 2:
            B[i] = np.array([1, p[j][0], p[j][1], p[j][2], p[j][0] * p[j][0], p[j][0] * p[j][1],
                          p[j][0] * p[j][2], p[j][1] * p[j][1], p[j][1] * p[j][2], p[j][2] * p[j][2]])
        W[i][i] = wendland(np.linalg.norm(x - p[j]))
    a = np.linalg.solve(np.dot(np.dot(B.T, W), B), np.dot(np.dot(B.T, W), d))
    
    if polyDegree == 0:
        Bx = np.array([1])
    elif polyDegree == 1:
        Bx = np.array([1, x[0], x[1], x[2]])
    elif polyDegree == 2:
        Bx = np.array([1, x[0], x[1], x[2], x[0] * x[0], x[0] * x[1], x[0] * x[2], x[1] * x[1], x[1] * x[2], x[2] * x[2]])
    return np.dot(Bx, a)

In [22]:
#Compute implicit function
fx_optional = np.array([implicit_function_optional(xi) for xi in x])

In [23]:
# Marcing tet to extract surface
sv_optional, sf_optional, _, _ = igl.marching_tets(x, T, fx_optional, 0)
sf_index_optional = igl.face_components(sf_optional)
num_dict_optional = {}
for num in sf_index_optional:
    num_dict_optional[num] = num_dict_optional[num] + 1 if num in num_dict_optional.keys() else 1
max_index_optional = max(num_dict_optional,key=num_dict_optional.get)
sf_new_optional = np.array([sf_optional[i] for i, index in enumerate(sf_index_optional) if index == max_index_optional])
mp.plot(sv_optional, sf_new_optional, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(22.531221…